# Number recognition neural network

## Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
import random

## Prepare training and test data

In [ ]:
# Load the data
(train_X, train_y), (test_X, test_y) = mnist.load_data()

imageSize = 28
numImages = 60000
# Normalize data
(train_X, train_y), (test_X, test_y) = (train_X/256, train_y), (test_X/256, test_y)
# Plot the first image
plt.xticks([])
plt.yticks([])
plt.imshow( train_X[0], cmap='gray' )
plt.show()

In [ ]:
# Plot the first 35 images
for i in range(35):
    plt.subplot(5, 7, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow( train_X[i], cmap='gray' )

plt.show()

In [ ]:
# Add additional value '1' to the beginning of each image (necessary for the bias)
zeros = np.full([numImages, 1], 1)
zeros1 = np.full([10000, 1], 1)
train_X = train_X.reshape(60000, 28*28)
test_X = test_X.reshape(10000, 28*28)
train_X = np.append(zeros, train_X, axis= 1)
test_X = np.append(zeros1, test_X, axis= 1)
print("train_X shape: " + str(train_X.shape))
print("test_X shape: " + str(test_X.shape))

### Transform training labels array to array of 10 component vectors

In [ ]:
# Data preview
train_y[0:10]

In [ ]:
# Transform the numbers to 10 component vectors
train_y = np.identity(10)[train_y]
# Preview after tranformation
train_y[0:10]

## Setting up the neural network

Input layer length: 785 neurons (1 bias + 28x28 pixels)

Hidden layer length: 10 neurons (this number can be changed if needed to impove performance)

Output layer length: 10 neurons (result catergories - 10 digits)

In [ ]:
# Define the number of layers and neurons per layer
numLayers = [28*28+1,10,10]

### Initialize weights: Numbers in the range from -2 to 2

In [ ]:
# Generate random weights for layers input-hidden and hidden-output
weights = [
    4 * np.random.random_sample([numLayers[0],numLayers[1]]) - 2,
    4 * np.random.random_sample([numLayers[1],numLayers[2]]) - 2
    ]

### Activation function

In [ ]:
# A monotonuous function that rescales a value to the range [0,1] (in this case sigmoid function) and it's derivative
def activation(x):
    return 1 / (1 + np.exp(-x))

def activationDerivative(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

# Plot functions
x = np.linspace(-5,5,101)
y1 = activation(x)
y2 = activationDerivative(x)
plt.plot(x, y1, label='activation function a(x)')
plt.plot(x, y2, label="derivative a'(x)")
plt.legend()
plt.show()

### Calculate output of the neural network

The value of a neuron is given as the dot product of the two vectors:

    weights
    output of the neurons in the previous layer

This value is then rescaled by the activation function.

In [ ]:
# A function that calculates the output of a layer taking input or output from previous layer and weights
def calculateNext(x,w):
    # x: input
    # w: weights
    return activation( np.dot(x,w) )

In [ ]:
# Apply the function twice to directly calculate the output - 10 component vector
testIndex = 50
calculateNext(
    calculateNext( train_X[testIndex], weights[0] ),
    weights[1]
)

In [ ]:
# Given an array of values returns an array of 10 component vectors
calculateNext(
    calculateNext( train_X, weights[0] ),
    weights[1]
)

### Calculate accuracy and individual error

In [ ]:
# Accuracy: calculate percentage of correctly guessed targets
def accuracy(testIn,testOut,weights):
    return (len(testOut) - np.sum(
            np.abs(
                np.sign(
                    np.abs(testOut - calculateNext(calculateNext(testIn, weights[0]), weights[1]).argmax(axis=1))
                )
            )
        )) / len(testOut)

In [ ]:
# Test the function on an array of 10 inputs and 10 target values
acc = accuracy(test_X[0:10],test_y[0:10],weights)
print("Prediction: " + str(calculateNext(calculateNext(test_X[0:10], weights[0]), weights[1]).argmax(axis=1)))
print("Target:     " + str(test_y[0:10]))
print("Accuracy: " + str(acc))

In [ ]:
# Error: calculate the difference between each pair of 10 component vectors (predicted and target)
# Subtract vectors, transform all values to absolute values and return the sum of those values
def error(predictedValues, correctValues):
    print(predictedValues)
    print(correctValues)
    print(np.sum( np.absolute(predictedValues - correctValues) ))
    return np.sum( np.absolute(predictedValues - correctValues) )

In [ ]:
# Test the function with one 10 component vector
error(calculateNext(calculateNext( train_X[testIndex], weights[0] ),weights[1]), train_y[testIndex])

### Calculate gradient 

In [ ]:
# Slow calculation: Calculate the gradient function using loops (10000 steps per 1100s)
def gradientSlow(x,w,correctValues):
    dot = np.dot(x,w[0])
    hiddenValues = activation( dot ) 
    hiddenValuesDerivative = activationDerivative( dot )
    hiddenValuesDerivative2 = activationDerivative(np.dot(hiddenValues,w[1]))
    diff = calculateNext(hiddenValues, w[1]) - correctValues

    grad1 = np.array([
        2 * diff * hiddenValuesDerivative2 * hiddenValues[i]
        for i in range(numLayers[1])])
    grad0 = np.array([
        [ 2 * np.sum(
            diff[:] * hiddenValuesDerivative2[:] * w[1][i,:] * hiddenValuesDerivative[i] * x[k]
        ) for i in range(numLayers[1])] 
        for k in range(numLayers[0])])

    return [grad0, grad1]

In [ ]:
# Faster calculation: using matrix multiplication and only one loop (10000 steps per 10s)
def gradient(x,w,correctValues):
    dot = np.dot(x,w[0])
    hiddenValues = activation( dot ) 
    #hiddenValues = calculateNext( x, w[0] )
    hiddenValuesDerivative = activationDerivative( dot )
    hiddenValuesDerivative2 = activationDerivative(np.dot(hiddenValues,w[1]))
    diff = calculateNext(hiddenValues, w[1]) - correctValues

    grad1 = 2 * np.matmul(np.transpose(np.array([hiddenValues])), np.array([diff * hiddenValuesDerivative2]) )
    grad0 = 2 * np.sum(np.array([np.matmul(np.transpose(np.array([x])),
            np.array([diff[j] * hiddenValuesDerivative2[j] * w[1][:,j] * hiddenValuesDerivative]))
            for j in range(numLayers[2])]), axis=0)

    return [grad0, grad1]

In [ ]:
# Fastets calculation: only matrix multiplication without loops (10000 steps per 6s)
def gradientFast(x,w,correctValues):
    dot = np.dot(x,w[0])
    hiddenValues = activation( dot ) 
    #hiddenValues = calculateNext( x, w[0] )
    hiddenValuesDerivative = activationDerivative( dot )
    hiddenValuesDerivative2 = activationDerivative(np.dot(hiddenValues,w[1]))
    diff = calculateNext(hiddenValues, w[1]) - correctValues
    
    grad1 = 2 * np.matmul(np.transpose(np.array([hiddenValues])), np.array([diff * hiddenValuesDerivative2]) )
    grad0 = 2 * np.matmul(np.transpose(np.array([x])), np.array([(np.matmul(np.transpose(diff*hiddenValuesDerivative2),(np.transpose(w[1]) * hiddenValuesDerivative)))]))

    return [grad0, grad1]

In [ ]:
# Test if matrix multiplication has been done correctly
gradSlow = gradientSlow(train_X[testIndex], weights, train_y[testIndex])
grad = gradient(train_X[testIndex], weights, train_y[testIndex])
gradFast = gradientFast(train_X[testIndex], weights, train_y[testIndex])
print("First number in grad0 for all 3 methods: ")
print(gradSlow[0][0][0], grad[0][0][0], gradFast[0][0][0])
print("First number in grad1 for all 3 methods: ")
print(gradSlow[1][0][0], grad[1][0][0], gradFast[1][0][0])

## Training: Use Gradient descent to change weights to minimize the error

Repeat the following process many time:

    Select a random image (index)
    Calculate the gradient of the error
    Change weights accoding to:

newWeights=oldWeights−learningRate⋅Gradient

In [ ]:
# Define learningRate and steps
learningRate = 0.01
steps = 100000

# For quality check keep track of errors in each step
errorList = [error(calculateNext(calculateNext(train_X[testIndex], weights[0]), weights[1]), train_y[testIndex])]

In [ ]:
# Train the model for number of steps
for i in range(steps):
    # pick random input
    index = np.random.randint(numImages)
    # update weights (go along opposite gradient)
    grad = gradientFast(train_X[index], weights, train_y[index])
    weights[0] = weights[0] - learningRate*grad[0]
    weights[1] = weights[1] - learningRate*grad[1]
    # calculate new error
    er = error(calculateNext(calculateNext(train_X[index], weights[0]), weights[1]), train_y[index])
    errorList.append( er )

In [ ]:
# Plot the errors for each step
plt.scatter(range(steps+1),errorList)

## Test the model on new data (test set)

In [ ]:
# Preview the data
for i in range(35):
    plt.subplot(5, 7, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow( test_X[i,1:].reshape(imageSize, imageSize), cmap='gray' )

plt.show()

In [ ]:
# Make a prediction for the first picture (7) and plot the output
testIndex = 0
calculateNext(calculateNext(test_X[testIndex], weights[0]), weights[1])
plt.bar(range(10), calculateNext(calculateNext(test_X[testIndex], weights[0]), weights[1])
)

In [ ]:
# Compare the output and the target value
print("Prediction:   " + str(calculateNext(calculateNext(test_X[testIndex], weights[0]), weights[1]).argmax()))
print("Target value: " + str(test_y[testIndex]))

In [ ]:
# Compare the first 35 cases of output and target values from the test set
print("Predictions:   ")
print(calculateNext(calculateNext(test_X, weights[0]), weights[1]).argmax(axis=1)[:35].reshape(5,7))
print("Target values: ")
print(test_y[:35].reshape(5,7))

In [ ]:
# Calculate and compare accuracy across train and test sets
print("Accuracy on train set: " + str(accuracy(train_X,np.argmax(train_y, axis=1),weights)))
print("Accuracy on test set : " + str(accuracy(test_X,test_y,weights)))